##추천 시스템의 목적
 - 컨텐츠 제공 플랫폼 내 아이템 소비 증대

 ex) 구글 플레이스토어 40%의 어플이, 유튜브 총 시청시간의 60%가 추천 시스템으로부터 적용 

###용어 정리
1. item(아이템=컨텐츠) : 추천할 객체
2. Query : 추천을 위한 모든 정보
 - User-Query(유저쿼리) : ID, 과거 구매이력(좋아요), 별점 등등 - 개인적인 정보
 - Additional-Query(추가쿼리) : 위치, 이용시간, 검색기록 - 사회적인 정보
3. Embeding(임베딩) : 쿼리의 정보를 수치화하여 담은 공간, 물리적 개념X

###추천시스템 문제해결 순서 흐름도
1. 후보생성 알고리즘 
 - 컨텐츠 기반 추천시스템(**아이템간의 유사도** 이용)
 - 협업 기반 추천시스템(**유저, 아이템간의 상관관계** 이용)
2. 점수화
 - 추가 알고리즘 적용 from 업로드 시간, 구독자 수, 시청시간 등
3. 서열화
 - 점수화 한 컨텐츠들을 필터링하여 점수 순 정렬

##컨텐츠 기반 추천 시스템
영화 추천 알고리즘
- 컨텐츠 기반 추천 시스템 -> 추천 아이템의 순수 속성에 의존

ex) 
1. 영화 인셉션에 '좋아요' 표현
2. 영화 데이터베이스 검색
3. 인셉션과 유사한 영화 검색 후 추천

key point) 인셉션과 가장 유사한 영화 검색

###코사인 유사도(cosine similarity)

벡터 내적으로 구함 $cosθ = \frac{\vec A · \vec B}{|\vec A| · |\vec B|}$ 유사한 경우 1, 상극인 경우 -1

- 코드

def cosine_similarity(a,b):

    num = np.dot(a,b)

    den = np.sqrt(np.sum(a**2))*np.sqrt(np.sum(b**2))

    return num/(den+1e-15)

- 따라서 영화 추천 방법

(드라마, 공포, 액션, S.F.)

1. 인셉션의 특징 벡터 추출 (0,1,4,10)
2. 다른 영화의 특징 벡터 추출 1번(10,0,0,0), 2번(1,1,7,3), 3번(2,0,0,10), 4번(0,10,1,0)
3. 인셉션과 다른 모든 영화들간의 코사인 유사도 계산
4. 3의 결과 중 가장 높은 영화 추천

(1번:어바웃타임, 2번:다크나이트, 3번:마션, 4번:더넌)


In [ ]:
import numpy as np

def cosine_similarity(a,b):
  num = np.dot(a,b)
  den = np.sqrt(np.sum(a**2))*np.sqrt(np.sum(b**2))
  return num/(den+1e-15)


In [ ]:
incep = np.array([0,1,4,10]) # 기준

#데이터베이스 내 영화 특징

about_time = np.array([10,0,0,0])
darknight = np.array([1,1,7,3])
martian = np.array([2,0,0,10])
thenun = np.array([0,10,1,0])

In [ ]:
#인셉션과 영화 코사인 유사도

movies = [about_time,darknight,martian,thenun]
names = ['about_time','darknight','martian','thenun']

scores = []
for mvec in movies:
  sim = cosine_similarity(incep,mvec)
  scores.append(sim)

In [ ]:
# 유사도 높은 순으로 정렬
idx = np.argsort(scores) #크기순 정렬 함수
recom = np.array(names)[idx]
print(np.flip(recom))

['martian' 'darknight' 'thenun' 'about_time']


##여기까지 알 수 있는 컨텐츠 기반 추천 시스쳄의 장단점
###장점
1. 유저와 유저간의 관계가 독립적, 대용량 데이터셋에 적합
2. 특별한 취향, 소수의 특별한 취향 존중

###단점
1. 모든 컨텐츠에대한 정보를 파악하고 있어야함
2. 시스템에 등록되고 기록이 있는 유저에게만 적용가 능한 시스템

##협업 기반 추천 시스템

###아이디어
(성별, 액션성, 연령대) : embedding(원소), 정규화 완료

U1 = (-1, 0.8, -0.5) : 여성, 액션 영화 좋아함, 연령대는 어린편

U2 = (-1, -0.5, 0.3)

U3 = (1, 0.6, 0.7)

U4 = (1, 0.2, 0.9) : 남성, 액션 영화를 싫어하진않음, 연령대는 높은 편

M1 = (0, 0.3, 0.6) : 성별에 구애받지않음, 액션성 약간 있음, 연령대가 있는 분들이 좋아함

M2 = (-0.6, -1, -0.1)

M3 = (0.6, 0.9, 0.5) : 남성들이 좋아하는 편, 액션영화, 연령대가 어느정도 있음

M4 = (0.3, -0.5, -0.7)

M5 = (0.25, 0.2, 0.86)

$U =
\begin{pmatrix}
U1 \\
U2 \\
U3 \\
U4
\end{pmatrix} , U \in R^{m*d} $ m은 유저 수, ,d는 embedding

$V =
\begin{pmatrix}
M1 \\
M2 \\
M3 \\
M4 \\
M5
\end{pmatrix} , U \in R^{n*d} $ m은 item 수, ,d는 embedding

$UV^{T} \in R^{m*n}$

###협업기반 추천시스템 => 유저가 선호하는 특징, 영화가 지닌 특징을 뭉침
###학습단계
$ Loss = \sum (A_{i,j}-UV^{T})^{2} + \sum (UV^{T})^{2} $
- 추론단계
 1. 새 유저의 embedding 추출
 2. 가장 비슷한 embedding 지닌 유저 검색


 ## 장단점
 
 - 장점
 1. 아이템 컨텐츠 자체를 학습할 필요가 없음
 2. 새로운 취향의 아이템을 추천하는 경우 존재
 
 - 단점
 1. 새로 추가될 아이템에 대한 부분이 없을 시 문제가 됨
 2. 가지고있던 임베딩의 정보 이외 새로운 경우의 수를 생각하기 어려움 